In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

/Users/leesu/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28*28
n_noise = 128 # 생성자의 입력값으로 사용할 노이즈의 크기

In [3]:
# GAN은 비지도 학습이므로 Y를 사용하지 않는다.
# 구분자에 넣을 이미지가 실제 이미지와 생성한 가짜 이미지 두 개이고, 가짜 이미지는 노이즈에서 생성할 것이므로 Z는 노이즈를 입력받음.
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise]) 

### 생성자 신경망

생성자는 무작위로 생성한 노이즈를 받아 가중치와 편향을 반영하여 은닉층을 만들고 은닉층에서 실제 이미지와 같은 크기의 결과값을 출력한다

In [4]:
# 은닉층으로 출력하기 위한 가중치와 편향
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))

# 출력층에 사용할 가중치와 편향 - 실제 이미지의 크기와 같아야 한다(28*28)
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

In [5]:
def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
    return output

### 구분자 신경망

0~1 사이의 스칼라값 하나를 출력하기 위해 활성화 함수로 sigmoid 함수 사용

In [6]:
# 구분자는 진짜와 얼마나 가까운가를 판단하는 값으로 0~1 사이의 값을 출력한다(스칼라)
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1])) # 스칼라

In [7]:
def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
    return output

In [8]:
# 노이즈 생성 함수
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [9]:
# 노이즈 Z를 이용해 가짜 이미지를 만들 생성자 G를 만들고, 
# G가 만든 가짜 이미지와 진짜 이미지 X를 각각 구분자에 넣어 입력한 이미지가 진짜인지 판별한다
G = generator(Z)
D_gene = discriminator(G)
D_real = discriminator(X)

### Loss Function

생성자가 만든 이미지를 구분자가 가짜라고 판단하도록 하는 손실값과 진짜라고 판단하도록 하는 손실값을 구해야 한다.

진짜 이미지 판별값 D_real은 1에 가까워야 하고(진짜라고 판별), 가짜 이미지 판별값 D_gene는 0에 가까워야 한다(가짜라고 판별)

In [10]:
# D_real과 1에서 D_gene를 뺀 값을 더한 값을 손실값으로 하여 이 값을 최대화
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))

# 가짜 이미지 판별값 D_gene를 1에 가깝게 만듬(가짜 이미지를 넣어도 진짜같다고 판별)
loss_G = tf.reduce_mean(tf.log(D_gene))

* GAN의 학습은 loss_D와 loss_G 모두를 최대화하는 것이다. loss_D가 증가하려면 loss_G는 하락해야 하고, loss_G가 증가하려면 loss_D가 하락해야 하는 등 경쟁 관계로 서로 연관되어 있기 때문에 두 손실값이 항상 같이 증가하는 경향을 보이지는 않는다.

### Training

In [11]:
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

In [12]:
# loss를 최대화하기 위해 minimize한 값에 마이너스를 붙여 최적화한다.
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0,0

In [14]:
import os

directory = 'samples' # 결과 저장할 디렉토리
if not os.path.exists(directory):
    os.makedirs(directory)

In [15]:
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        # 배치 크기만큼 노이즈를 만들어서 생성자의 입력값으로
        noise = get_noise(batch_size, n_noise)
        
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_xs, Z: noise}) # 구분자는 X 값을 받는다
        _, loaa_val_G = sess.run([train_G, loss_G], feed_dict={Z: noise}) # 생성자는 노이즈Z 값을 받는다
        
    print('Epoch:', '%04d' % epoch, 'D loss: {:.4f}'.format(loss_val_D), 'G loss: {:.4f}'.format(loss_val_G))
    
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})

        # 28*28 크기의 가짜 이미지로 만들어 samples 폴더에 저장
        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))

        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))

        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)

Epoch: 0000 D loss: -0.5789 G loss: 0.0000
Epoch: 0001 D loss: -0.2375 G loss: 0.0000
Epoch: 0002 D loss: -0.1497 G loss: 0.0000
Epoch: 0003 D loss: -0.4627 G loss: 0.0000
Epoch: 0004 D loss: -0.3646 G loss: 0.0000
Epoch: 0005 D loss: -0.3677 G loss: 0.0000
Epoch: 0006 D loss: -0.4504 G loss: 0.0000
Epoch: 0007 D loss: -0.5881 G loss: 0.0000
Epoch: 0008 D loss: -0.2394 G loss: 0.0000
Epoch: 0009 D loss: -0.5208 G loss: 0.0000
Epoch: 0010 D loss: -0.4454 G loss: 0.0000
Epoch: 0011 D loss: -0.4954 G loss: 0.0000
Epoch: 0012 D loss: -0.3910 G loss: 0.0000
Epoch: 0013 D loss: -0.3526 G loss: 0.0000
Epoch: 0014 D loss: -0.4034 G loss: 0.0000
Epoch: 0015 D loss: -0.3961 G loss: 0.0000
Epoch: 0016 D loss: -0.4424 G loss: 0.0000
Epoch: 0017 D loss: -0.4683 G loss: 0.0000
Epoch: 0018 D loss: -0.3831 G loss: 0.0000
Epoch: 0019 D loss: -0.3999 G loss: 0.0000
Epoch: 0020 D loss: -0.4470 G loss: 0.0000
Epoch: 0021 D loss: -0.5189 G loss: 0.0000
Epoch: 0022 D loss: -0.4113 G loss: 0.0000
Epoch: 0023